In [ ]:
# =========================
# Link Ingestion MVP (Notebook-Ready)
# =========================
# deps: openai, trafilatura, bs4, lxml or html5lib, python-dotenv, pandas, requests
# usage (quick start at bottom of this cell)
import os, re, json, datetime, typing as T
from dataclasses import dataclass
from urllib.parse import urlparse
from pathlib import Path

import requests
import pandas as pd
from bs4 import BeautifulSoup
import trafilatura
from dotenv import load_dotenv


In [ ]:

# ---- Load environment (expects OPENAI_API_KEY) ----
load_dotenv()
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:


# =========================
# Config (edit as you like)
# =========================
MODEL_WITH_WEB = os.getenv("MODEL_WITH_WEB", "gpt-4o-mini")     # has web tool on eligible accounts
MODEL_FALLBACK = os.getenv("MODEL_FALLBACK", "gpt-4.1-mini")    # local LLM fallback
TAXONOMY_PATH  = os.getenv("TAXONOMY_PATH", "taxonomy.json")    # where allowed lists persist
CSV_PATH       = os.getenv("CSV_PATH", "links_store.csv")       # where your DataFrame persists

# MVP

## Function 1

In [ ]:

DEFAULT_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:128.0) Gecko/20100101 Firefox/128.0"
}

STRICT_JSON_RULES = (
    "Return STRICT JSON with keys: "
    '["title","author","publish_date","categories","tags","tldr","language","citations","confidence_notes"]. '
    "categories=1–3 short labels; tags=5–12 keywords; tldr=3–6 crisp bullets; "
    "language=two-letter ISO code; citations=list of {title,url}; "
    "confidence_notes=1–2 short sentences. No markdown—JSON only."
)

# =========================
# Types
# =========================
@dataclass
class PageContent:
    url: str
    domain: str
    title: T.Optional[str]
    author: T.Optional[str]
    publish_date: T.Optional[str]
    text: str
    html_len: int
    text_len: int

# =========================
# Helpers: parsing & fetching
# =========================
def _best_bs4_parser():
    try:
        import lxml  # noqa
        return "lxml"
    except Exception:
        try:
            import html5lib  # noqa
            return "html5lib"
        except Exception:
            return "html.parser"

def _safe_json_loads(text: str) -> dict:
    try:
        return json.loads(text)
    except Exception:
        m = re.search(r"\{.*\}", text, re.S)
        return json.loads(m.group(0)) if m else {"error": "Non-JSON output", "raw": text[:1200]}

def _fetch_html(url: str) -> str:
    r = requests.get(url, headers=DEFAULT_HEADERS, timeout=25)
    r.raise_for_status()
    return r.text

def _guess_meta(soup: BeautifulSoup) -> dict:
    meta = {"title": None, "author": None, "publish_date": None}
    # title
    if soup.title and soup.title.string:
        meta["title"] = soup.title.string.strip()
    ogt = soup.find("meta", property="og:title")
    if ogt and ogt.get("content"):
        meta["title"] = ogt["content"].strip()
    # author
    for k in ["author", "article:author", "og:article:author"]:
        m = soup.find("meta", attrs={"name": k}) or soup.find("meta", attrs={"property": k})
        if m and m.get("content"):
            meta["author"] = m["content"].strip(); break
    # date
    for k in ["article:published_time", "og:published_time", "pubdate", "publish-date", "date"]:
        m = soup.find("meta", attrs={"name": k}) or soup.find("meta", attrs={"property": k})
        if m and m.get("content"):
            meta["publish_date"] = m["content"].strip(); break
    if meta["publish_date"] is None:
        t = soup.find("time")
        if t and (t.get("datetime") or t.text):
            meta["publish_date"] = (t.get("datetime") or t.text).strip()
    return meta

def extract_readable_text(url: str) -> PageContent:
    html = _fetch_html(url)
    text = (trafilatura.extract(html, include_comments=False, include_tables=False, favor_precision=True) or "").strip()

    if len(text) < 200:
        PARSER = _best_bs4_parser()
        soup = BeautifulSoup(html, PARSER)
        for tag in soup(["script","style","noscript","header","footer","nav","aside"]):
            tag.extract()
        text = soup.get_text("\n", strip=True)

    PARSER = _best_bs4_parser()
    soup_full = BeautifulSoup(html, PARSER)
    meta = _guess_meta(soup_full)

    text = re.sub(r"\n{3,}", "\n\n", text)
    return PageContent(
        url=url,
        domain=urlparse(url).netloc,
        title=meta["title"],
        author=meta["author"],
        publish_date=meta["publish_date"],
        text=text,
        html_len=len(html or ""),
        text_len=len(text or "")
    )

# =========================
# Taxonomy persistence
# =========================
def load_taxonomy(path: str = TAXONOMY_PATH) -> dict:
    p = Path(path)
    if not p.exists():
        return {"categories": [], "tags": []}
    with p.open("r", encoding="utf-8") as f:
        data = json.load(f)
    # ensure keys
    data.setdefault("categories", [])
    data.setdefault("tags", [])
    return data

def save_taxonomy(categories: T.List[str], tags: T.List[str], path: str = TAXONOMY_PATH) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump({"categories": categories, "tags": tags}, f, ensure_ascii=False, indent=2)

# =========================
# Matching & evolving lists
# =========================
def _normalize_token(s: str) -> str:
    return re.sub(r"\s+", " ", s.strip().lower())

def update_matches(candidates: T.List[str], allowed: T.List[str], max_k: int) -> T.Tuple[T.List[str], T.List[str]]:
    """
    Try to match candidates against allowed list.
    If no match found for a candidate, add it as a new allowed term.
    Returns (final_matches, updated_allowed_list).
    """
    allowed_norm = { _normalize_token(a): a for a in allowed }
    matches, updated_allowed = [], allowed.copy()
    seen = set()

    for c in candidates:
        if not isinstance(c, str):
            continue
        c_clean = c.strip()
        if not c_clean:
            continue
        c_norm = _normalize_token(c_clean)

        hit = None
        # exact
        if c_norm in allowed_norm:
            hit = allowed_norm[c_norm]
        else:
            # substring match either way
            for an_norm, raw in allowed_norm.items():
                if c_norm in an_norm or an_norm in c_norm:
                    hit = raw; break
        if hit:
            if hit not in seen:
                matches.append(hit); seen.add(hit)
        else:
            # new term → append to allowed
            if c_clean not in updated_allowed:
                updated_allowed.append(c_clean)
            if c_clean not in seen:
                matches.append(c_clean); seen.add(c_clean)

        if len(matches) >= max_k:
            break

    return matches, updated_allowed

# =========================
# LLM calls (web tool → local fallback)
# =========================
def analyze_link_with_web_tool(url: str, allowed_categories: T.List[str], allowed_tags: T.List[str]) -> dict:
    if not hasattr(client, "responses"):
        raise RuntimeError("responses_api_unavailable")

    user_prompt = (
        f"{STRICT_JSON_RULES}\n\n"
        f"When labeling categories and tags, prefer from the lists below when semantically appropriate.\n"
        f"Allowed categories: {json.dumps(allowed_categories[:50])}\n"
        f"Allowed tags: {json.dumps(allowed_tags[:200])}\n\n"
        f"Read this URL and summarize with citations: {url}"
    )

    resp = client.responses.create(
        model=MODEL_WITH_WEB,
        tools=[{"type": "web_search"}],
        input=[
            {"role": "system", "content": "You are a precise analyst that reads the provided URL using the web tool."},
            {"role": "user", "content": user_prompt}
        ],
    )
    output_text = getattr(resp, "output_text", getattr(resp, "output", ""))
    return _safe_json_loads(output_text)

def summarize_local_content(page: PageContent, allowed_categories: T.List[str], allowed_tags: T.List[str]) -> dict:
    payload = {
        "source_url": page.url,
        "detected_title": page.title,
        "detected_author": page.author,
        "detected_publish_date": page.publish_date,
        "article_text": page.text[:22_000],
        "allowed_categories": allowed_categories[:50],
        "allowed_tags": allowed_tags[:200]
    }
    if hasattr(client, "responses"):
        resp = client.responses.create(
            model=MODEL_FALLBACK,
            input=[
                {"role": "system", "content": STRICT_JSON_RULES},
                {"role": "user", "content": json.dumps(payload)}
            ],
        )
        output_text = getattr(resp, "output_text", getattr(resp, "output", ""))
        return _safe_json_loads(output_text)
    else:
        # very old fallback
        messages = [
            {"role": "system", "content": STRICT_JSON_RULES},
            {"role": "user", "content": json.dumps(payload)},
        ]
        cc = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
        return _safe_json_loads(cc.choices[0].message.content)

# =========================
# Public API: main function
# =========================
def analyze_link_plus(
    url: str,
    allowed_categories: T.List[str] = None,
    allowed_tags: T.List[str] = None,
    force_local: bool = False,
) -> dict:
    """
    Returns a normalized record (dict) and includes updated taxonomy under _taxonomy.
    Keys:
        fetched_at_utc, url, domain, headline, categories, tags, tldr (list),
        content_text, source_title, author, publish_date, _source, _taxonomy
    """
    assert url.startswith("http"), "Pass a valid http(s) URL."
    allowed_categories = allowed_categories or []
    allowed_tags = allowed_tags or []

    # parse locally first so you can verify and fall back if needed
    page = extract_readable_text(url)

    # LLM metadata
    try:
        if force_local:
            raise RuntimeError("forced_local")
        llm_data = analyze_link_with_web_tool(url, allowed_categories, allowed_tags)
        mode = "openai_web_tool"
        model_used = MODEL_WITH_WEB
    except Exception:
        if page.text_len < 200:
            raise RuntimeError("Could not extract enough text; page may be paywalled or script-rendered.")
        llm_data = summarize_local_content(page, allowed_categories, allowed_tags)
        mode = "local_fallback"
        model_used = MODEL_FALLBACK

    # normalize fields
    fetched_at_utc = datetime.datetime.utcnow().isoformat() + "Z"
    raw_categories = llm_data.get("categories") or []
    raw_tags = llm_data.get("tags") or []
    tldr = llm_data.get("tldr") or []

    # TLDR as list
    if isinstance(tldr, str):
        parts = [p.strip(" -•\t") for p in re.split(r"[\n•\-]+", tldr) if p.strip()]
        tldr = parts[:6] if parts else [llm_data.get("tldr", "")]

    # evolve taxonomy
    categories, updated_categories = update_matches(raw_categories, allowed_categories, max_k=3)
    tags, updated_tags = update_matches(raw_tags, allowed_tags, max_k=12)

    record = {
        "fetched_at_utc": fetched_at_utc,
        "url": page.url,
        "domain": page.domain,
        "headline": (llm_data.get("title") or page.title or "").strip()[:300],
        "categories": categories,
        "tags": tags,
        "tldr": tldr,
        "content_text": page.text,    # full parsed text so you can verify LLM output
        "source_title": page.title,
        "author": llm_data.get("author") or page.author,
        "publish_date": llm_data.get("publish_date") or page.publish_date,
        "_source": {"mode": mode, "model": model_used},
        "_taxonomy": {
            "updated_categories": updated_categories,
            "updated_tags": updated_tags
        }
    }
    return record

# =========================
# DataFrame Store (in-memory + CSV)
# =========================
COLUMNS = [
    "fetched_at_utc","url","domain","headline","categories","tags","tldr",
    "content_text","source_title","author","publish_date"
]

def init_store() -> pd.DataFrame:
    return pd.DataFrame(columns=COLUMNS)

def append_record(df: pd.DataFrame, record: dict) -> pd.DataFrame:
    # store lists as Python lists (dtype=object)
    row = {k: record.get(k, None) for k in COLUMNS}
    return pd.concat([df, pd.DataFrame([row])], ignore_index=True)

def save_csv(df: pd.DataFrame, path: str = CSV_PATH) -> None:
    df2 = df.copy()
    for col in ["categories","tags","tldr"]:
        if col in df2.columns:
            df2[col] = df2[col].apply(lambda x: x if isinstance(x, str) else json.dumps(x, ensure_ascii=False))
    df2.to_csv(path, index=False)

def load_csv(path: str = CSV_PATH) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        return init_store()
    df = pd.read_csv(path)
    for col in ["categories","tags","tldr"]:
        if col in df.columns:
            df[col] = df[col].apply(lambda s: json.loads(s) if isinstance(s, str) and s.startswith("[") else [])
    return df

def simple_search(df: pd.DataFrame, query: str, top_n: int = 50) -> pd.DataFrame:
    q = (query or "").lower().strip()
    if not q:
        return df.head(top_n)
    mask = (
        df["headline"].fillna("").str.lower().str.contains(q) |
        df["url"].fillna("").str.lower().str.contains(q) |
        df["domain"].fillna("").str.lower().str.contains(q) |
        df["content_text"].fillna("").str.lower().str.contains(q) |
        df["tags"].astype(str).str.lower().str.contains(q) |
        df["categories"].astype(str).str.lower().str.contains(q)
    )
    return df[mask].head(top_n)



In [ ]:
## Store record incrementally in the CSV

# =========================
# URL canonicalization (for stable dedupe)
# =========================
from urllib.parse import urlparse, parse_qsl, urlencode, urlunparse

_STRIP_PARAMS = {
    "utm_source","utm_medium","utm_campaign","utm_term","utm_content",
    "utm_name","utm_id","gclid","gclsrc","fbclid","mc_cid","mc_eid"
}

def canonicalize_url(url: str) -> str:
    """
    Normalize URL for dedupe:
    - lowercase scheme/host
    - remove common tracking params
    - sort remaining query params
    - strip trailing slash (except root)
    """
    parsed = urlparse(url)
    scheme = (parsed.scheme or "https").lower()
    netloc = parsed.netloc.lower()
    path = parsed.path or "/"
    # normalize trailing slash
    if path != "/" and path.endswith("/"):
        path = path[:-1]

    # clean & sort query
    q = []
    for k, v in parse_qsl(parsed.query, keep_blank_values=True):
        if k.lower() in _STRIP_PARAMS:
            continue
        q.append((k, v))
    q.sort(key=lambda kv: kv[0].lower())
    query = urlencode(q, doseq=True)

    canon = urlunparse((scheme, netloc, path, "", query, ""))
    return canon

# =========================
# DataFrame Store (CSV cache with dedupe by url_canonical)
# =========================
COLUMNS = [
    "fetched_at_utc","url","url_canonical","domain","headline",
    "categories","tags","tldr","content_text","source_title","author","publish_date"
]

def init_store() -> pd.DataFrame:
    return pd.DataFrame(columns=COLUMNS)

def _ensure_columns(df: pd.DataFrame) -> pd.DataFrame:
    # add any missing columns (for backward compatibility)
    for col in COLUMNS:
        if col not in df.columns:
            df[col] = None
    # backfill url_canonical if missing/empty
    if df["url_canonical"].isna().any() or (df["url_canonical"] == "").any():
        df["url_canonical"] = df.apply(
            lambda r: canonicalize_url(r["url"]) if pd.isna(r["url_canonical"]) or r["url_canonical"] == "" else r["url_canonical"],
            axis=1
        )
    return df[COLUMNS]

def save_csv(df: pd.DataFrame, path: str = CSV_PATH) -> None:
    df2 = df.copy()
    # lists -> JSON strings for portability
    for col in ["categories","tags","tldr"]:
        if col in df2.columns:
            df2[col] = df2[col].apply(lambda x: x if isinstance(x, str) else json.dumps(x, ensure_ascii=False))
    df2.to_csv(path, index=False)

def load_csv(path: str = CSV_PATH) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        return init_store()
    df = pd.read_csv(path)
    # restore lists
    for col in ["categories","tags","tldr"]:
        if col in df.columns:
            df[col] = df[col].apply(lambda s: json.loads(s) if isinstance(s, str) and s.startswith("[") else ([] if pd.isna(s) else s))
    df = _ensure_columns(df)
    return df

def append_record(df: pd.DataFrame, record: dict) -> pd.DataFrame:
    row = {
        "fetched_at_utc": record.get("fetched_at_utc"),
        "url": record.get("url"),
        "url_canonical": canonicalize_url(record.get("url","")),
        "domain": record.get("domain"),
        "headline": record.get("headline"),
        "categories": record.get("categories"),
        "tags": record.get("tags"),
        "tldr": record.get("tldr"),
        "content_text": record.get("content_text"),
        "source_title": record.get("source_title"),
        "author": record.get("author"),
        "publish_date": record.get("publish_date"),
    }
    return pd.concat([df, pd.DataFrame([row])], ignore_index=True)

def get_cached_row(df: pd.DataFrame, url: str) -> T.Optional[dict]:
    canon = canonicalize_url(url)
    hits = df[df["url_canonical"] == canon]
    if len(hits) == 0:
        return None
    rec = hits.iloc[0].to_dict()
    # ensure list types for convenience
    for col in ["categories","tags","tldr"]:
        v = rec.get(col)
        if isinstance(v, str):
            try:
                rec[col] = json.loads(v) if v.startswith("[") else [v]
            except Exception:
                rec[col] = [v]
    return rec

# =========================
# Orchestrator: ingest OR fetch from cache
# =========================
def ingest_or_fetch(
    url: str,
    taxonomy_path: str = TAXONOMY_PATH,
    csv_path: str = CSV_PATH,
    force_reingest: bool = False,
) -> T.Tuple[dict, pd.DataFrame]:
    """
    If canonical URL exists in CSV, return the cached row (and skip LLM).
    Else run analyze_link_plus(), update taxonomy JSON, append 1 row to CSV, and return the new row.
    Returns: (row_as_dict, updated_df)
    """
    # 0) load CSV (cache) and taxonomy
    df = load_csv(csv_path)
    tax = load_taxonomy(taxonomy_path)
    tax.setdefault("categories", [])
    tax.setdefault("tags", [])

    # 1) cached?
    if not force_reingest:
        cached = get_cached_row(df, url)
        if cached is not None:
            return cached, df  # nothing else to do

    # 2) not cached → analyze
    rec = analyze_link_plus(
        url,
        allowed_categories=tax["categories"],
        allowed_tags=tax["tags"]
    )

    # 3) update taxonomy (evolving lists)
    tax["categories"] = rec["_taxonomy"]["updated_categories"]
    tax["tags"] = rec["_taxonomy"]["updated_tags"]
    save_taxonomy(tax["categories"], tax["tags"], taxonomy_path)

    # 4) append to CSV cache
    df = append_record(df, rec)
    save_csv(df, csv_path)

    # 5) return a compact dict consistent with CSV row formatting
    row = {
        "fetched_at_utc": rec["fetched_at_utc"],
        "url": rec["url"],
        "url_canonical": canonicalize_url(rec["url"]),
        "domain": rec["domain"],
        "headline": rec["headline"],
        "categories": rec["categories"],
        "tags": rec["tags"],
        "tldr": rec["tldr"],
        "content_text": rec["content_text"],
        "source_title": rec["source_title"],
        "author": rec["author"],
        "publish_date": rec["publish_date"],
    }
    return rec, row, df

# =========================
# Quick demo — mimics your preferred usage
# =========================
test_url = "https://www.confident-ai.com/blog/why-llm-as-a-judge-is-the-best-llm-evaluation-method"

rec, row, df = ingest_or_fetch(test_url)
# `row` is the single record (dict); `df` is the full accumulated store
# If you run again with the same URL, it will return from cache and skip LLM.

# show the row as a one-line DataFrame (your key fields)
pd.DataFrame([{
    "fetched_at_utc": row["fetched_at_utc"],
    "headline": row["headline"],
    "categories": row["categories"],
    "tags": row["tags"],
    "tldr": row["tldr"],
    "url": row["url"]
}])


/var/folders/wl/_gt22h7503dgw67xjtv9ccn00000gn/T/ipykernel_6815/3911997558.py:291: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  fetched_at_utc = datetime.datetime.utcnow().isoformat() + "Z"


,fetched_at_utc,headline,categories,tags,tldr,url
0,2025-09-13T15:07:09.608280Z,LLM-as-a-Judge Simply Explained: The Complete ...,"[AI/ML, Product, Startups]","[LLM, RAG, LangGraph, Agentic AI, Vector DB, P...","[LLM-as-a-Judge automates LLM evaluation, enha...",https://www.confident-ai.com/blog/why-llm-as-a...


In [ ]:
df

,fetched_at_utc,url,url_canonical,domain,headline,categories,tags,tldr,content_text,source_title,author,publish_date
0,2025-09-13T15:05:07.059924Z,https://medium.com/data-science/from-data-scie...,https://medium.com/data-science/from-data-scie...,medium.com,From Data Scientist to ML / AI Product Manager,"[AI/ML, Product, Career]","[ML Product Manager, AI Product Manager, Data ...",[Transitioning from Data Scientist to ML/AI Pr...,From Data Scientist to ML / AI Product Manager...,From Data Scientist to ML / AI Product Manager,Anna Via,2024-04-03
1,2025-09-13T15:07:09.608280Z,https://www.confident-ai.com/blog/why-llm-as-a...,https://www.confident-ai.com/blog/why-llm-as-a...,www.confident-ai.com,LLM-as-a-Judge Simply Explained: The Complete ...,"[AI/ML, Product, Startups]","[LLM, RAG, LangGraph, Agentic AI, Vector DB, P...","[LLM-as-a-Judge automates LLM evaluation, enha...",Your LLM app is streaming out tokens faster th...,LLM-as-a-Judge Simply Explained: The Complete ...,Jeffrey Ip,2025-08-21


## Function 2

In [ ]:
# =========================
# MVP Flashcards (Single CSV + Swipe Queue)
# =========================

import os, re, json, hashlib, datetime, typing as T
from pathlib import Path
from collections import deque

import pandas as pd
from dotenv import load_dotenv

# ---- OpenAI client ----
load_dotenv()
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# =========================
# Config
# =========================
CARDS_CSV_PATH = os.getenv("CARDS_CSV_PATH", "cards_store.csv")
MODEL_FOR_CARDS = os.getenv("MODEL_FOR_CARDS", "gpt-4.1-mini")  # lightweight, cheap
MAX_TEXT_CHARS = int(os.getenv("MAX_TEXT_CHARS", "8000"))       # cap article text sent to LLM

# =========================
# CSV schema
# =========================
CARD_COLUMNS = [
    "card_id",         # pk (sha1 of url_canonical + "\n" + normalized_question)
    "url_canonical",   # article key
    "question",
    "answer",
    "learned",         # bool
    "created_at_utc"   # ISO str
]

def _ensure_cards_csv(path: str = CARDS_CSV_PATH) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        df = pd.DataFrame(columns=CARD_COLUMNS)
        df.to_csv(path, index=False)
        return df
    df = pd.read_csv(path)
    # Backward/robust handling
    for col in CARD_COLUMNS:
        if col not in df.columns:
            df[col] = None
    # Coerce learned -> bool
    if "learned" in df.columns:
        df["learned"] = df["learned"].apply(lambda x: bool(x) if isinstance(x, (bool,)) else (str(x).lower() == "true"))
    return df[CARD_COLUMNS]

def _save_cards_df(df: pd.DataFrame, path: str = CARDS_CSV_PATH) -> None:
    df2 = df.copy()
    df2.to_csv(path, index=False)

# =========================
# Utilities
# =========================
def _normalize_question(q: str) -> str:
    q = (q or "").strip()
    # collapse spaces, strip trailing ?
    q = re.sub(r"\s+", " ", q)
    return q

def _card_id(url_canonical: str, question: str) -> str:
    base = f"{url_canonical}\n{_normalize_question(question)}"
    return hashlib.sha1(base.encode("utf-8")).hexdigest()[:16]

def _utc_now_iso() -> str:
    return datetime.datetime.utcnow().isoformat() + "Z"

def _safe_json_loads(text: str) -> dict:
    try:
        return json.loads(text)
    except Exception:
        m = re.search(r"\{.*\}", text, re.S)
        return json.loads(m.group(0)) if m else {"error": "Non-JSON", "raw": text[:1200]}

# =========================
# LLM: generate Q&A from content_text
# =========================
_SYSTEM_PROMPT = (
    "You generate concise, factual Q&A flashcards from text. Avoid trivia and ambiguity. "
    "Prefer concrete facts, key definitions, mechanisms, comparisons, and takeaways. "
    "Keep questions ≤ 140 characters, answers ≤ 240 characters."
)

_USER_TEMPLATE = """From the text below, produce 3–6 Q&A flashcards that capture the core facts/concepts.
Return STRICT JSON only:
{"cards":[{"q":"...","a":"..."}]}

Text (may be truncated):
{TEXT}
"""

def _call_llm_for_cards(content_text: str, model: str = MODEL_FOR_CARDS) -> T.List[T.Dict[str, str]]:
    # Use Responses API if available
    payload = _USER_TEMPLATE.replace("{TEXT}", (content_text or "")[:MAX_TEXT_CHARS])
    if hasattr(client, "responses"):
        resp = client.responses.create(
            model=model,
            input=[
                {"role": "system", "content": _SYSTEM_PROMPT},
                {"role": "user", "content": payload}
            ],
        )
        out = getattr(resp, "output_text", getattr(resp, "output", ""))
        data = _safe_json_loads(out)
    else:
        # very old fallback
        cc = client.chat.completions.create(
            model=model,
            messages=[{"role": "system", "content": _SYSTEM_PROMPT},
                      {"role": "user", "content": payload}]
        )
        data = _safe_json_loads(cc.choices[0].message.content)

    cards = data.get("cards", [])
    # lightweight validation & cleaning
    clean = []
    seen_q = set()
    for c in cards:
        q = _normalize_question(str(c.get("q", "")).strip())
        a = str(c.get("a", "")).strip()
        if not q or not a:
            continue
        if len(q) > 200 or len(a) > 500:
            # discard egregiously long pairs (model might go verbose)
            continue
        if q.lower() in seen_q:
            continue
        seen_q.add(q.lower())
        clean.append({"q": q, "a": a})
    return clean[:6]  # cap to 6

# =========================
# Public API
# =========================
def generate_cards_for_url(url_canonical: str, content_text: str, n: int = 5) -> pd.DataFrame:
    """
    Idempotent. Generates Q&A with LLM and upserts into cards_store.csv.
    Never flips learned=True back to False.
    Returns the full cards DataFrame (all cards).
    """
    if not url_canonical or not isinstance(url_canonical, str):
        raise ValueError("url_canonical must be a non-empty string.")

    df = _ensure_cards_csv(CARDS_CSV_PATH)
    # Ask LLM for cards
    pairs = _call_llm_for_cards(content_text)
    if not pairs:
        return df  # nothing to add

    # Build new rows; upsert by card_id
    existing_ids = set(df["card_id"].astype(str).tolist())
    rows_to_add = []
    now_iso = _utc_now_iso()

    count_added = 0
    for pair in pairs:
        if count_added >= max(1, n):  # ensure at least 1 if any generated
            break
        q, a = pair["q"], pair["a"]
        cid = _card_id(url_canonical, q)
        if cid in existing_ids:
            # don't overwrite; skip creating a dup
            continue
        rows_to_add.append({
            "card_id": cid,
            "url_canonical": url_canonical,
            "question": q,
            "answer": a,
            "learned": False,
            "created_at_utc": now_iso
        })
        count_added += 1

    if rows_to_add:
        df = pd.concat([df, pd.DataFrame(rows_to_add)], ignore_index=True)
        _save_cards_df(df, CARDS_CSV_PATH)
    return df

def load_unlearned_cards(url_canonical: str) -> pd.DataFrame:
    df = _ensure_cards_csv(CARDS_CSV_PATH)
    mask = (df["url_canonical"] == url_canonical) & (~df["learned"])
    return df[mask].copy()

def start_session(url_canonical: str, shuffle: bool = True) -> deque:
    """
    Returns a deque of dicts: [{"card_id","question","answer"}, ...] for unlearned cards.
    """
    df = load_unlearned_cards(url_canonical)
    print(f"Found {len(df)} unlearned cards for {url_canonical}")
    cards = df[["card_id","question","answer"]].to_dict(orient="records")
    if shuffle and len(cards) > 1:
        import random
        random.shuffle(cards)
    return deque(cards)

def swipe_left(queue: deque) -> None:
    """
    "Review again": move the current card to the end of the queue.
    No persistence change.
    """
    if not queue:
        return
    queue.append(queue.popleft())

def swipe_right(queue: deque, card_id: str) -> None:
    """
    "I know it": mark learned=True in CSV, then remove from the queue.
    """
    if not queue:
        return
    current = queue[0]
    if current["card_id"] != card_id:
        # guard: if caller passed mismatched card_id, align to current
        card_id = current["card_id"]

    # persist learned=True
    df = _ensure_cards_csv(CARDS_CSV_PATH)
    idx = df.index[df["card_id"] == card_id]
    if len(idx) > 0:
        df.loc[idx, "learned"] = True
        _save_cards_df(df, CARDS_CSV_PATH)

    # remove from queue
    queue.popleft()

def count_unlearned(url_canonical: str) -> int:
    return len(load_unlearned_cards(url_canonical))

def reset_learned(url_canonical: str) -> int:
    """
    Set learned=False for all cards of an article.
    Returns the number of cards reset.
    """
    df = _ensure_cards_csv(CARDS_CSV_PATH)
    mask = (df["url_canonical"] == url_canonical)
    n = int(mask.sum())
    if n > 0:
        df.loc[mask, "learned"] = False
        _save_cards_df(df, CARDS_CSV_PATH)
    return n

# =========================
# Mini-Demo (how to use)
# =========================
# Assume you already ingested an article and have:
#   - url_canonical from your canonicalize_url(url)
#   - content_text from your ingestion result (df['content_text'] or rec['content_text'])

# Example placeholders (replace with your real values):
# url_canonical_demo = canonicalize_url(rec["url"])  # if you have canonicalize_url() from your earlier code
# content_text_demo  = rec["content_text"]

# Generate cards (idempotent; safe to re-run)
# generate_cards_for_url(url_canonical_demo, content_text_demo, n=5)

# Start a learning session
# q = start_session(url_canonical_demo, shuffle=True)
# if not q:
#     print("No unlearned cards. You're done!")
# else:
#     # Peek at current card
#     cur = q[0]
#     print("Q:", cur["question"])
#     print("A:", cur["answer"])
#     # Simulate user action:
#     # swipe_left(q)  # if user wants to review again
#     # swipe_right(q, cur["card_id"])  # if user knows it


In [ ]:
url_canonical = "https://www.confident-ai.com/blog/why-llm-as-a-judge-is-the-best-llm-evaluation-method"
df_cards = generate_cards_for_url(url_canonical, row["content_text"], n=5)
df_cards

/var/folders/wl/_gt22h7503dgw67xjtv9ccn00000gn/T/ipykernel_6815/2457346259.py:70: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.datetime.utcnow().isoformat() + "Z"


,card_id,url_canonical,question,answer,learned,created_at_utc
0,1a8f61e63debc5a3,https://www.confident-ai.com/blog/why-llm-as-a...,What is LLM-as-a-Judge?,LLM-as-a-Judge uses large language models to e...,False,2025-09-13T15:55:11.385772Z
1,7a64dabeca7e0612,https://www.confident-ai.com/blog/why-llm-as-a...,What are the two main types of LLM-as-a-Judge?,Single-output scoring evaluates one output bas...,False,2025-09-13T15:55:11.385772Z
2,778d03e73c69b33d,https://www.confident-ai.com/blog/why-llm-as-a...,Why use LLM-as-a-Judge over human evaluation o...,"LLM-as-a-Judge is more scalable, accurate, and...",False,2025-09-13T15:55:11.385772Z
3,a2e35c695451c833,https://www.confident-ai.com/blog/why-llm-as-a...,What inputs are typically provided to an LLM j...,"The original input prompt, the generated outpu...",False,2025-09-13T15:55:11.385772Z
4,e614d7e6881a0c5e,https://www.confident-ai.com/blog/why-llm-as-a...,How does pairwise LLM-as-a-Judge work?,It compares two or more LLM outputs for the sa...,False,2025-09-13T15:55:11.385772Z
5,f3645bd8a4ed6a5f,https://www.confident-ai.com/blog/why-llm-as-a...,What is LLM-as-a-Judge used for?,LLM-as-a-Judge uses large language models to a...,False,2025-09-13T15:55:23.656486Z
6,ff443fcc9b3dbe54,https://www.confident-ai.com/blog/why-llm-as-a...,How does single-output referenceless judging w...,It scores one LLM output against a rubric with...,False,2025-09-13T15:55:23.656486Z
7,c9663654e2a81f19,https://www.confident-ai.com/blog/why-llm-as-a...,What is the advantage of reference-based singl...,It uses a gold-standard expected output as an ...,False,2025-09-13T15:55:23.656486Z
8,7c1cc5ad8cea707b,https://www.confident-ai.com/blog/why-llm-as-a...,Why is LLM-as-a-Judge gaining popularity over ...,"LLM judges are scalable, faster, and better ca...",False,2025-09-13T15:55:23.656486Z
9,5e4579936c3e0505,https://www.confident-ai.com/blog/why-llm-as-a...,What are common use cases for LLM-as-a-Judge?,It evaluates single-turn or multi-turn LLM int...,False,2025-09-13T15:55:23.656486Z


In [ ]:

while q:
    card = q[0]


KeyboardInterrupt: 

In [50]:
q = start_session(url_canonical, shuffle=True)
q

deque([{'card_id': 'c9663654e2a81f19',
        'question': 'What is the advantage of reference-based single-output judging?',
        'answer': 'It uses a gold-standard expected output as an anchor to improve evaluation consistency and reproducibility when scoring LLM outputs.'},
       {'card_id': '1a8f61e63debc5a3',
        'question': 'What is LLM-as-a-Judge?',
        'answer': 'LLM-as-a-Judge uses large language models to evaluate LLM outputs by scoring them against custom criteria such as accuracy, relevance, and bias.'},
       {'card_id': 'ff443fcc9b3dbe54',
        'question': 'How does single-output referenceless judging work?',
        'answer': 'It scores one LLM output against a rubric without a gold-standard answer, useful for open-ended tasks by evaluating accuracy, coherence, and helpfulness independently.'},
       {'card_id': '7c1cc5ad8cea707b',
        'question': 'Why is LLM-as-a-Judge gaining popularity over human evaluators and traditional metrics?',
        'answ

In [53]:
# === Button-based swipe UI (requires ipywidgets) ===
# If needed: pip install ipywidgets && jupyter nbextension enable --py widgetsnbextension
from ipywidgets import Button, HBox, VBox, HTML, Output
from IPython.display import display, clear_output

def run_widget_session(url_canonical, shuffle=True):
    queue = start_session(url_canonical, shuffle=shuffle)
    out = Output()

    if not queue:
        display(HTML("<b>No unlearned cards for this article. 🎉</b>"))
        return

    # widgets
    q_html = HTML("<h4>Question</h4>")
    a_html = HTML("<i>(click 'Reveal answer')</i>")
    a_shown = {"val": False}

    btn_reveal = Button(description="Reveal answer", button_style="")
    btn_know   = Button(description="I know it 👍", button_style="success")
    btn_review = Button(description="Review again 🔁", button_style="warning")

    # helpers
    def render_current():
        if not queue:
            with out:
                clear_output()
                display(HTML("<b>All cards learned for this article. 🎉</b>"))
            # disable buttons
            btn_reveal.disabled = True
            btn_know.disabled   = True
            btn_review.disabled = True
            return

        card = queue[0]
        q_html.value = f"<h4>Question</h4><div>{card['question']}</div>"
        a_html.value = "<i>(click 'Reveal answer')</i>"
        a_shown["val"] = False

    def on_reveal(_):
        if not queue:
            return
        card = queue[0]
        a_html.value = f"<b>Answer</b><div style='margin-top:6px;'>{card['answer']}</div>"
        a_shown["val"] = True

    def on_know(_):
        if not queue:
            return
        card = queue[0]
        swipe_right(queue, card["card_id"])   # persist learned=True and pop from queue
        render_current()

    def on_review(_):
        if not queue:
            return
        swipe_left(queue)                     # move current to end
        render_current()

    btn_reveal.on_click(on_reveal)
    btn_know.on_click(on_know)
    btn_review.on_click(on_review)

    with out:
        clear_output()
        render_current()
        display(VBox([
            q_html,
            a_html,
            HBox([btn_reveal, btn_know, btn_review])
        ]))

    display(out)

# usage:
# run_widget_session(url_canonical, shuffle=True)


In [ ]:
run_widget_session(url_canonical, shuffle=True)

In [ ]:
# === CLI-style swipe loop (works in any notebook) ===
def run_text_session(url_canonical):
    q = start_session(url_canonical, shuffle=True)
    if not q:
        print("No unlearned cards for this article. 🎉")
        return

    print(f"Session loaded with {len(q)} cards. Type:\n"
          "  s = show answer, r = I know it (remove), l = review again (send to end), q = quit\n")

    while q:
        card = q[0]
        print("\nQ:", card["question"])
        cmd = input("Command [s/r/l/q]: ").strip().lower()

        if cmd == "q":
            print("Exited.")
            break
        elif cmd == "s":
            print("A:", card["answer"])
        elif cmd == "r":
            swipe_right(q, card["card_id"])
            print("✅ Marked learned. Remaining:", len(q))
        elif cmd == "l":
            swipe_left(q)
            print("🔁 Moved to end. Queue size:", len(q))
        else:
            print("Unknown command. Use s/r/l/q.")

    if not q:
        print("\nAll cards learned for this article. 🎉")




In [ ]:
# usage:
run_text_session(url_canonical)

# Archive

## Function1

In [ ]:
# # === deps (assumes you've already installed: openai, trafilatura, bs4, lxml or html5lib, python-dotenv) ===
# import os, re, json, datetime, typing as T
# from dataclasses import dataclass, asdict
# from urllib.parse import urlparse

# import requests
# from bs4 import BeautifulSoup
# import trafilatura
# from dotenv import load_dotenv

# load_dotenv()
# from openai import OpenAI
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# # ------------ Parser selection (robust) ------------
# def _best_bs4_parser():
#     try:
#         import lxml  # noqa: F401
#         return "lxml"
#     except Exception:
#         try:
#             import html5lib  # noqa: F401
#             return "html5lib"
#         except Exception:
#             return "html.parser"

# # ------------ Models & settings ------------
# MODEL_WITH_WEB = "gpt-4o-mini"   # has web tool on supported accounts
# MODEL_FALLBACK  = "gpt-4.1-mini" # summarization if web tool unavailable

# STRICT_JSON_RULES = (
#     "Return STRICT JSON with keys: "
#     '["title","author","publish_date","categories","tags","tldr","language","citations","confidence_notes"]. '
#     "categories=1–3 short labels; tags=5–12 keywords; tldr=3–6 crisp bullets; "
#     "language=two-letter ISO code; citations=list of {title,url}; "
#     "confidence_notes=1–2 short sentences. No markdown—JSON only."
# )

# DEFAULT_HEADERS = {
#     "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:128.0) Gecko/20100101 Firefox/128.0"
# }

# @dataclass
# class PageContent:
#     url: str
#     domain: str
#     title: T.Optional[str]
#     author: T.Optional[str]
#     publish_date: T.Optional[str]
#     text: str
#     html_len: int
#     text_len: int
#     _html: T.Optional[str] = None

# # ------------ Helpers ------------
# def _safe_json_loads(text: str) -> dict:
#     try:
#         return json.loads(text)
#     except Exception:
#         m = re.search(r"\{.*\}", text, re.S)
#         return json.loads(m.group(0)) if m else {"error": "Non-JSON output", "raw": text[:1200]}

# def _fetch_html(url: str) -> str:
#     r = requests.get(url, headers=DEFAULT_HEADERS, timeout=25)
#     r.raise_for_status()
#     return r.text

# def _guess_meta(soup: BeautifulSoup) -> dict:
#     meta = {"title": None, "author": None, "publish_date": None}
#     # title
#     if soup.title and soup.title.string:
#         meta["title"] = soup.title.string.strip()
#     ogt = soup.find("meta", property="og:title")
#     if ogt and ogt.get("content"):
#         meta["title"] = ogt["content"].strip()
#     # author
#     for k in ["author", "article:author", "og:article:author"]:
#         m = soup.find("meta", attrs={"name": k}) or soup.find("meta", attrs={"property": k})
#         if m and m.get("content"):
#             meta["author"] = m["content"].strip(); break
#     # date
#     for k in ["article:published_time", "og:published_time", "pubdate", "publish-date", "date"]:
#         m = soup.find("meta", attrs={"name": k}) or soup.find("meta", attrs={"property": k})
#         if m and m.get("content"):
#             meta["publish_date"] = m["content"].strip(); break
#     if meta["publish_date"] is None:
#         t = soup.find("time")
#         if t and (t.get("datetime") or t.text):
#             meta["publish_date"] = (t.get("datetime") or t.text).strip()
#     return meta

# def extract_readable_text(url: str, include_html: bool=False) -> PageContent:
#     html = _fetch_html(url)

#     # 1) trafilatura (best effort)
#     text = (trafilatura.extract(
#         html, include_comments=False, include_tables=False, favor_precision=True
#     ) or "").strip()

#     # 2) fallback: BeautifulSoup
#     if len(text) < 200:
#         PARSER = _best_bs4_parser()
#         soup = BeautifulSoup(html, PARSER)
#         for tag in soup(["script","style","noscript","header","footer","nav","aside"]):
#             tag.extract()
#         text = soup.get_text("\n", strip=True)

#     PARSER = _best_bs4_parser()
#     soup_full = BeautifulSoup(html, PARSER)
#     meta = _guess_meta(soup_full)

#     text = re.sub(r"\n{3,}", "\n\n", text)
#     return PageContent(
#         url=url,
#         domain=urlparse(url).netloc,
#         title=meta["title"],
#         author=meta["author"],
#         publish_date=meta["publish_date"],
#         text=text,
#         html_len=len(html or ""),
#         text_len=len(text or ""),
#         _html=html if include_html else None
#     )

# # ------------ Web tool path (no response_format to be version-agnostic) ------------
# def analyze_link_with_web_tool(url: str) -> dict:
#     if not hasattr(client, "responses"):
#         raise RuntimeError("responses_api_unavailable")

#     base_kwargs = dict(
#         model=MODEL_WITH_WEB,
#         tools=[{"type": "web_search"}],
#         input=[
#             {"role": "system", "content": "You are a precise analyst that reads the provided URL using the web tool."},
#             {"role": "user", "content": f"{STRICT_JSON_RULES}\n\nRead this URL and summarize it with citations: {url}"}
#         ],
#     )
#     resp = client.responses.create(**base_kwargs)
#     # Some SDKs expose .output_text, others .output; handle both:
#     output_text = getattr(resp, "output_text", getattr(resp, "output", ""))
#     return _safe_json_loads(output_text)

# # ------------ Fallback summarization (local content + LLM) ------------
# def summarize_local_content(page: PageContent) -> dict:
#     payload = {
#         "source_url": page.url,
#         "detected_title": page.title,
#         "detected_author": page.author,
#         "detected_publish_date": page.publish_date,
#         "article_text": page.text[:22_000]
#     }
#     if hasattr(client, "responses"):
#         resp = client.responses.create(
#             model=MODEL_FALLBACK,
#             input=[
#                 {"role": "system", "content": STRICT_JSON_RULES},
#                 {"role": "user", "content": json.dumps(payload)}
#             ],
#         )
#         output_text = getattr(resp, "output_text", getattr(resp, "output", ""))
#         return _safe_json_loads(output_text)
#     else:
#         # Very old SDK fallback via Chat Completions
#         messages = [
#             {"role": "system", "content": STRICT_JSON_RULES},
#             {"role": "user", "content": json.dumps(payload)},
#         ]
#         cc = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
#         return _safe_json_loads(cc.choices[0].message.content)

# # ------------ Public API returning BOTH LLM + content ------------
# def analyze_link_plus(url: str, include_html: bool=False, text_cap: int=50_000) -> dict:
#     """
#     Returns:
#     {
#       "llm": {...},                # categories/tags/tldr/etc.
#       "content": {
#           "url","domain","meta":{"title","author","publish_date"},
#           "text","text_len","html_len", ["html" if include_html=True]
#       },
#       "_source": {...}             # mode + bookkeeping
#     }
#     """
#     assert url.startswith("http"), "Pass a valid http(s) URL."

#     # Always build content locally so you can verify tags/summary
#     page = extract_readable_text(url, include_html=include_html)
#     content_block = {
#         "url": page.url,
#         "domain": page.domain,
#         "meta": {"title": page.title, "author": page.author, "publish_date": page.publish_date},
#         "text": page.text[:text_cap],                   # cap returned text to keep memory manageable
#         "text_len": page.text_len,
#         "html_len": page.html_len,
#     }
#     if include_html and page._html is not None:
#         content_block["html"] = page._html  # careful: can be huge

#     # Try web tool first for the LLM output
#     try:
#         llm_data = analyze_link_with_web_tool(url)
#         mode = "openai_web_tool"
#         model_used = MODEL_WITH_WEB
#     except Exception:
#         if page.text_len < 200:
#             raise RuntimeError("Could not extract enough text; page may be paywalled or script-rendered.")
#         llm_data = summarize_local_content(page)
#         # fill missing fields
#         llm_data.setdefault("title", page.title)
#         llm_data.setdefault("author", page.author)
#         llm_data.setdefault("publish_date", page.publish_date)
#         llm_data.setdefault("citations", [{"title": page.title or "", "url": page.url}])
#         mode = "local_fallback"
#         model_used = MODEL_FALLBACK

#     result = {
#         "llm": llm_data,
#         "content": content_block,
#         "_source": {
#             "mode": mode,
#             "model": model_used,
#             "url": page.url,
#             "domain": page.domain,
#             "fetched_at": datetime.datetime.utcnow().isoformat() + "Z"
#         }
#     }
#     return result


# test_url = "https://www.confident-ai.com/blog/why-llm-as-a-judge-is-the-best-llm-evaluation-method"
# out = analyze_link_plus(test_url, include_html=False, text_cap=40000)
# print(json.dumps(out["llm"], indent=2, ensure_ascii=False))
# print("---\nCONTENT META:", json.dumps(out["content"]["meta"], indent=2, ensure_ascii=False))
# print("TEXT PREVIEW:\n", out["content"]["text"])

In [ ]:
# # =========================
# # Quick smoke test / demo (Notebook-friendly)
# # =========================

# # 1) load taxonomy (or init if empty)
# tax = load_taxonomy(TAXONOMY_PATH)
# if not tax["categories"]:
#     tax["categories"] = ["AI/ML","Product","Startups","Career","Health","Finance"]
# if not tax["tags"]:
#     tax["tags"] = ["LLM","RAG","LangGraph","Agentic AI","Vector DB","Prompting","A/B Testing","Retention"]
# save_taxonomy(tax["categories"], tax["tags"], TAXONOMY_PATH)

# # 2) test a URL
# test_url = "https://www.confident-ai.com/blog/why-llm-as-a-judge-is-the-best-llm-evaluation-method"

# rec = analyze_link_plus(
#     test_url,
#     allowed_categories=tax["categories"],
#     allowed_tags=tax["tags"]
# )

# # 3) update taxonomy (so new categories/tags are remembered)
# tax["categories"] = rec["_taxonomy"]["updated_categories"]
# tax["tags"] = rec["_taxonomy"]["updated_tags"]
# save_taxonomy(tax["categories"], tax["tags"], TAXONOMY_PATH)

# # 4) convert to DataFrame directly
# import pandas as pd
# df = pd.DataFrame([{
#     "fetched_at_utc": rec["fetched_at_utc"],
#     "headline": rec["headline"],
#     "categories": rec["categories"],
#     "tags": rec["tags"],
#     "tldr": rec["tldr"],
#     "content_text": rec["content_text"],
#     "url": rec["url"]
# }])

# df


/var/folders/wl/_gt22h7503dgw67xjtv9ccn00000gn/T/ipykernel_6815/3911997558.py:291: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  fetched_at_utc = datetime.datetime.utcnow().isoformat() + "Z"


,fetched_at_utc,headline,categories,tags,tldr,content_text,url
0,2025-09-13T15:06:49.176125Z,LLM-as-a-Judge Simply Explained: The Complete ...,"[AI/ML, Product, Startups]","[LLM, RAG, LangGraph, Agentic AI, Vector DB, P...","[LLM-as-a-Judge automates LLM evaluation, enha...",Your LLM app is streaming out tokens faster th...,https://www.confident-ai.com/blog/why-llm-as-a...
